# Выявление особенностей словоупотребления в текстах Л. Н. Толстого методами дистрибутивной семантики

In [1]:
import os
import re
from lxml import etree
from pymystem3 import Mystem
import string
import nltk
from langdetect import detect

## Сегментатор

In [2]:
language_punkt_vars = nltk.tokenize.punkt.PunktLanguageVars
language_punkt_vars.sent_end_chars=('.','?','!','»','…')
with open('.\\data\\traindata.txt', 'r', encoding='utf-8') as f:
    traindata = f.read()
trainer = nltk.tokenize.punkt.PunktTrainer(traindata, language_punkt_vars)
trainer.INCLUDE_ALL_COLLOCS = True 
trainer.INCLUDE_ABBREV_COLLOCS = True
params = trainer.get_params()
sbd = nltk.tokenize.punkt.PunktSentenceTokenizer(params)

  Abbreviation: [1.1075] руб
  Abbreviation: [0.5093] матф
  Abbreviation: [1.5052] ул
  Abbreviation: [0.5537] стр
  Abbreviation: [4.0916] м
  Abbreviation: [4.0916] г
  Abbreviation: [1.5052] ст
  Abbreviation: [5.5374] тыс
  Abbreviation: [6.0209] т.п
  Abbreviation: [4.0916] н
  Abbreviation: [4.0916] п
  Abbreviation: [19.5680] см
0.0004105933073290905 0.008888888888888889 0.0 4871 225 2 2
0.0004105933073290905 0.008888888888888889 0.0 4871 225 2 2
0.003284746458632724 0.008888888888888889 0.003013344812742144 4871 225 16 2
0.010264832683227263 0.044444444444444446 0.008609556607834697 4871 225 50 10
0.003284746458632724 0.008888888888888889 0.003013344812742144 4871 225 16 2
0.0012317799219872716 0.008888888888888889 0.0008609556607834697 4871 225 6 2
0.00390063641962636 0.008888888888888889 0.003659061558329746 4871 225 19 2
0.000821186614658181 0.008888888888888889 0.00043047783039173483 4871 225 4 2
0.001642373229316362 0.02666666666666667 0.00043047783039173483 4871 225 8 6


## Данные

На основе 90-томного собрания сочинений

In [3]:
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [4]:
mystem = Mystem()

In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
def preprocessing_1(text):
    text = re.sub('[0-9—A-Za-zÉéÂâÊêÎîÔôÛûÀàÈèÙùËëÏïÜüŸÿÇç«»’]+','', text)
    text = re.sub('[\n\s]+',' ', text)
    text.replace('[','').replace(']','')
    text = sbd.sentences_from_text(text, realign_boundaries=True)
    return text

In [16]:
def preprocessing_2(sentences, f):
    for text in sentences:
        if langid.classify(text)[0] not in ['fr','la'] and len(text)>25:
            text = text.translate(text.maketrans('','',string.punctuation))
            text = ''.join(mystem.lemmatize(text.strip()))
            f.write(text)

In [8]:
import langid

In [9]:
langid.classify("Радуйтесь когда мир вас оскорбляет и гонит за правду так же гнали и пророков")

('ru', -1461.9347658157349)

### Произведения

** Плохие теги: **

- note
- add
- sic
- orig
- head
- del
- figure
- table


** Ненужные теги: **

- pb
- hi
- corr
- reg
- choice
- div
- p

In [10]:
tags_for_deletion  = ['.//tei:note',
                      './/tei:add',
                      './/tei:sic',
                      './/tei:orig',
                      './/tei:head',
                      './/tei:del',
                      './/tei:figure',
                      './/tei:table'
                     ]

tegs_to_strip = ['{http://www.tei-c.org/ns/1.0}pb',
                '{http://www.tei-c.org/ns/1.0}hi',
                '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg',
                '{http://www.tei-c.org/ns/1.0}choice',
                '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div',
                '{http://www.tei-c.org/ns/1.0}lb']

In [11]:
def get_one_writing(filename):
    root = etree.parse(filename)#.getroot()
    for i in tags_for_deletion:
        for j in root.findall(i, tei_ns):
            parent = j.find("..")
            parent.remove(j)
    for i in tegs_to_strip:
        etree.strip_tags(root, i)
    return root.find(".//tei:text", tei_ns).text.strip()

In [13]:
get_one_writing('.\\data\\Final\\[«Ежели бы человек не желал…»] 1.xml')[:100]

'1) Ежели бы человек не желал, то и не было бы человека. Причина всякой деятельности есть желание. Же'

In [ ]:
mystem = Mystem()
preprocessing_2(preprocessing_1(get_one_writing('.\\data\\Final\\Abrégé de l’Evangile 24.xml')),f)

In [107]:
langid.classify('Et en parlant ainsi неразобр')[0]
preprocessing_1('Et en parlant ainsi неразобр')

['неразобр']

In [65]:
len('[Иоанна V,] 27, 28.')

19

In [12]:
path = '.\\data\\Final\\'

In [13]:
from datetime import datetime

In [ ]:
mystem = Mystem()
with open ('result.txt', 'w', encoding='utf-8') as f:
    for root, dirs, files in os.walk (path):
            for fl in files :
                start = datetime.now() 
                preprocessing_2(preprocessing_1(get_one_writing(path+fl)), f)
                print ('{}\t{}\t{}'.format(os.path.getsize(path+fl)//1024, 
                                          datetime.now()-start,
                                          fl))
                #print ('finished: ', datetime.now()-start, ' '+fl)
                #print (preprocessing_1(get_one_writing(path+fl)))

### Письма

### Дневники

## Обработка текста

### Теги

In [28]:
import nltk

In [19]:
a = 'Весной 1820 года Пушкина вызвали к военному генерал-губернатору Петербурга графу М. А. Милорадовичу для объяснения по поводу содержания его стихотворений (в том числе эпиграмм на Аракчеева, архимандрита Фотия и самого Александра I), несовместимых со статусом государственного чиновника. Шла речь о его высылке в Сибирь или заточении в Соловецкий монастырь. Лишь благодаря хлопотам друзей, прежде всего Карамзина, удалось добиться смягчения наказания. Его перевели из столицы на юг в кишинёвскую канцелярию И. Н. Инзова[34]. По пути к новому месту службы Александр Сергеевич заболевает воспалением лёгких, искупавшись в Днепре. Для поправления здоровья Раевские вывозят в конце мая 1820 года больного поэта с собой на Кавказ и в Крым[35]. По дороге семья Раевских и А. С. Пушкин останавливаются в г. Таганроге, в бывшем доме градоначальника П. А. Папкова (ул. Греческая, 40).'

In [21]:
a = 'Он находится на ул. Пушкина дом 17'

In [150]:
nltk.tokenize.sent_tokenize(tobetokenized, language='czech')

['Тогда Rambler Media (контролируется "Проф-медиа" Владимира Потанина) объявил о продаже Google 100% системы контекстной рекламы ЗАО "Бегун" за $140 млн (см.',
 'Ъ от 19 июля 2008 года).',
 'Всю жизнь доказывала, что она – лучшая, ноги стирая в кровь.',
 'За 22 года турне Павлова проехала по миру более полумиллиона километров, дала около 9 тыс.',
 'спектаклей.',
 'Так, например, «умышленное причинение смерти другому человеку», конечно же, – преступление, убийство (ст.',
 '105 УК РФ), но и… подвиг на войне.']

In [29]:
language_punkt_vars = nltk.tokenize.punkt.PunktLanguageVars
language_punkt_vars.sent_end_chars=('.','?','!','»','…')

In [166]:
print(nltk.tokenize.punkt.PunktLanguageVars.sent_end_chars)

('.', '?', '!', '»', '…')


In [30]:
with open('.\\data\\traindata.txt', 'r', encoding='utf-8') as f:
    traindata = f.read()
trainer = nltk.tokenize.punkt.PunktTrainer(traindata, language_punkt_vars)
trainer.INCLUDE_ALL_COLLOCS = True 
trainer.INCLUDE_ABBREV_COLLOCS = True
params = trainer.get_params()
sbd = nltk.tokenize.punkt.PunktSentenceTokenizer(params)

  Abbreviation: [1.1180] руб
  Abbreviation: [4.1305] н
  Abbreviation: [19.7537] см
  Abbreviation: [1.5195] ул
  Abbreviation: [1.5195] ст
  Abbreviation: [4.1305] м
  Abbreviation: [6.0781] т.п
  Abbreviation: [4.1305] г
  Abbreviation: [5.5900] тыс
  Abbreviation: [4.1305] п
  Abbreviation: [0.5590] стр
0.00043010752688172043 0.009615384615384616 0.0 4650 208 2 2
0.00043010752688172043 0.009615384615384616 0.0 4650 208 2 2
0.0034408602150537634 0.009615384615384616 0.0031517334533993696 4650 208 16 2
0.01032258064516129 0.04807692307692308 0.008554705087798289 4650 208 48 10
0.0034408602150537634 0.009615384615384616 0.0031517334533993696 4650 208 16 2
0.0012903225806451613 0.009615384615384616 0.0009004952723998199 4650 208 6 2
0.003870967741935484 0.009615384615384616 0.0036019810895992796 4650 208 18 2
0.0008602150537634409 0.009615384615384616 0.00045024763619990995 4650 208 4 2
0.0017204301075268817 0.028846153846153848 0.00045024763619990995 4650 208 8 6
0.003010752688172043 

In [168]:
with open('.\\data\\testdata.txt', 'r', encoding='utf-8') as f:
    tobetokenized = f.read()
for sentence in sbd.sentences_from_text(tobetokenized, realign_boundaries=True):
    print (sentence + '\n')

Тогда Rambler Media (контролируется "Проф-медиа" Владимира Потанина) объявил о продаже Google 100% системы контекстной рекламы ЗАО "Бегун" за $140 млн (см. Ъ от 19 июля 2008 года).

Всю жизнь доказывала, что она – лучшая, ноги стирая в кровь.

За 22 года турне Павлова проехала по миру более полумиллиона километров, дала около 9 тыс. спектаклей.

Так, например, «умышленное причинение смерти другому человеку», конечно же, – преступление, убийство (ст. 105 УК РФ), но и… подвиг на войне.



In [163]:
nltk.tokenize.sent_tokenize(tobetokenized, language='czech')

['Тогда Rambler Media (контролируется "Проф-медиа"',
 'Владимира Потанина) объявил о продаже Google 100% системы контекстной рекламы ЗАО "Бегун"',
 'за $140 млн (см.',
 'Ъ от 19 июля 2008 года).',
 'Всю жизнь доказывала, что она – лучшая, ноги стирая в кровь.',
 'За 22 года турне Павлова проехала по миру более полумиллиона километров, дала около 9 тыс.',
 'спектаклей.',
 'Так, например, «умышленное причинение смерти другому человеку», конечно же, – преступление, убийство (ст.',
 '105 УК РФ), но и… подвиг на войне.']

# Try

In [6]:
import gensim

In [15]:
sentences = gensim.models.word2vec.LineSentence('result.txt')

In [13]:
sentences = [['я', 'иду'], ['я', 'иду']]

In [ ]:
model = gensim.models.Word2Vec(sentences, min_count=1)